In [1]:
import pandas as pd
import pyodbc
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import date, datetime

In [2]:
ref_data = date.today()
ref_timestamp = datetime.now()

In [3]:
server = 'localhost'
database = 'db_datasus'
username = 'dev'
password = 'SenhaDev1234'

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password)

cursor = conn.cursor()

sql_query = pd.read_sql_query("""SELECT [date] 
                                     ,[state] 
                                     ,SUM([confirmed]) AS [confirmed] 
                                     ,SUM([deaths]) AS [deaths] 
                                     ,[estimated_population_2019] 
                                     ,[estimated_population] 
                                     ,[city_ibge_code] 
                                     ,[confirmed_per_100k_inhabitants] 
                                     ,[death_rate] 
                                FROM [db_datasus].[dbo].[COVID] 
                                WHERE city_ibge_code < 100 AND CONVERT(DATE, [date]) = CONVERT(DATE,GETDATE()-1) 
                                GROUP BY  
                                      [date] 
                                      ,[state] 
                                      ,[estimated_population_2019] 
                                      ,[estimated_population] 
                                      ,[city_ibge_code] 
                                      ,[confirmed_per_100k_inhabitants] 
                                      ,[death_rate] 
                                ORDER BY [date], [state] ASC""", conn)

sql_query

,date,state,confirmed,deaths,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-02-25,SP,1,0,45919049,46289333,35,0.0,0
1,2020-02-26,SP,1,0,45919049,46289333,35,0.0,0
2,2020-02-27,SP,1,0,45919049,46289333,35,0.0,0
3,2020-02-28,SP,2,0,45919049,46289333,35,0.0,0
4,2020-02-29,SP,2,0,45919049,46289333,35,0.0,0
...,...,...,...,...,...,...,...,...,...
10815,2021-04-21,RO,207030,4939,1777225,1796460,11,11524.0,0
10816,2021-04-21,RS,934970,23690,11377239,11422973,43,8185.0,0
10817,2021-04-21,SC,863842,12842,7164788,7252502,42,11911.0,0
10818,2021-04-21,SE,193271,4034,2298696,2318822,28,8335.0,0


In [4]:
df = pd.DataFrame(sql_query)
df

,date,state,confirmed,deaths,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-02-25,SP,1,0,45919049,46289333,35,0.0,0
1,2020-02-26,SP,1,0,45919049,46289333,35,0.0,0
2,2020-02-27,SP,1,0,45919049,46289333,35,0.0,0
3,2020-02-28,SP,2,0,45919049,46289333,35,0.0,0
4,2020-02-29,SP,2,0,45919049,46289333,35,0.0,0
...,...,...,...,...,...,...,...,...,...
10815,2021-04-21,RO,207030,4939,1777225,1796460,11,11524.0,0
10816,2021-04-21,RS,934970,23690,11377239,11422973,43,8185.0,0
10817,2021-04-21,SC,863842,12842,7164788,7252502,42,11911.0,0
10818,2021-04-21,SE,193271,4034,2298696,2318822,28,8335.0,0


In [5]:
df.to_parquet('df.parquet.snappy',compression='snappy')
pd.read_parquet('df.parquet.snappy')

,date,state,confirmed,deaths,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-02-25,SP,1,0,45919049,46289333,35,0.0,0
1,2020-02-26,SP,1,0,45919049,46289333,35,0.0,0
2,2020-02-27,SP,1,0,45919049,46289333,35,0.0,0
3,2020-02-28,SP,2,0,45919049,46289333,35,0.0,0
4,2020-02-29,SP,2,0,45919049,46289333,35,0.0,0
...,...,...,...,...,...,...,...,...,...
10815,2021-04-21,RO,207030,4939,1777225,1796460,11,11524.0,0
10816,2021-04-21,RS,934970,23690,11377239,11422973,43,8185.0,0
10817,2021-04-21,SC,863842,12842,7164788,7252502,42,11911.0,0
10818,2021-04-21,SE,193271,4034,2298696,2318822,28,8335.0,0


In [6]:
table = pa.Table.from_pandas(df)

In [7]:
pq.write_to_dataset(table, root_path='C:/Users/tiago/Desktop/'+str(ref_data))